<a href="https://colab.research.google.com/github/subuppaluru/LSTM/blob/main/LSTM_TextGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [3]:
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [6]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  163779
Total Vocab:  58


In [7]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  163679


In [8]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#Model

In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
#filename = "weights-improvement-19-1.9435.hdf5"
#model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [15]:
filepath = "weights-improvement-19-1.9435.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


In [16]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 256)               264192    
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 58)                14906     
                                                                 
Total params: 279,098
Trainable params: 279,098
Non-trainable params: 0
_________________________________________________________________


In [17]:
# fit the model
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1279/1279 [==============================] - ETA: 0s - loss: 2.9908
Epoch 00001: loss improved from inf to 2.99080, saving model to weights-improvement-19-1.9435.hdf5
1279/1279 [==============================] - 672s 524ms/step - loss: 2.9908
Epoch 2/20
1279/1279 [==============================] - ETA: 0s - loss: 2.8164
Epoch 00002: loss improved from 2.99080 to 2.81643, saving model to weights-improvement-19-1.9435.hdf5
1279/1279 [==============================] - 678s 530ms/step - loss: 2.8164
Epoch 3/20
1279/1279 [==============================] - ETA: 0s - loss: 2.7260
Epoch 00003: loss improved from 2.81643 to 2.72598, saving model to weights-improvement-19-1.9435.hdf5
1279/1279 [==============================] - 679s 531ms/step - loss: 2.7260
Epoch 4/20
1279/1279 [==============================] - ETA: 0s - loss: 2.6551
Epoch 00004: loss improved from 2.72598 to 2.65512, saving model to weights-improvement-19-1.9435.hdf5
1279/1279 [==============================] - 678

In [20]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [22]:
# pick a random seed
import sys
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" it's high time you were all in bed!'
on various pretexts they all moved off, and alice was soon left "
 the was oo and toene to ae infe to ae inre, and the wai qooe th the was ani the was oate anr the was oo and the tam oh the war of the war oate and the was oo and the tame th the waide 
ani eer a little sooell to ced to the waite and thet iar fare and the waite  and tou thit io a lange hireer, and the weile taid to the woide ' 
'ih i dan't teee the borme, in a cet ' said alice, ''that s tee sai ' sheue taed to herself, ''what so seke the girse wornd ' she seitg rabbit aare lere anrce. 
'ie iou't thte the moce turtle so ae in a cat,' said the caterpillar.

'ieve you gad to tae ' said the manch hare.

'ie d dene to teee ' said the monen, and the wait on an aeree an an ffreen, and the waited thth theee th the waale 'thet io thr the wai if a cotr of the sare an the crele, 
'the cir tat tha kart wireg ' shi seidg tert ontelely, 'in a lerg hf wou toeek the woid ' 
'                 